Do in shell :

`mkdir tmpzip && mkdir tmp && mkdir data && cd data`

`wget -r --no-parent https://data.cquest.org/inpi_rncs/imr/`

`mkdir data.cquest.org/inpi_rncs/imr/flux`

`mv data.cquest.org/inpi_rncs/imr/20* data.cquest.org/inpi_rncs/imr/flux/`

les flux sont ok mis il faut les dezippé : `for f in ./*7z; do 7z x $f; done`
supprimer tous les fichiers  md5

Passons aux stocks :

2017:
`for f in *.zip; do mkdir ${f:0:4} && mkdir ${f:0:4}/${f:5:2} && unzip $f -d ${f:0:4}/${f:5:2}/; done && rm *.zip && rm *.html;`

2018 ok

2019, 2020, 2021, 2022
`for f in */*/*.zip; do mkdir ${f:0:2}/${f:3:2}/${f:6:4} && mkdir ${f:0:2}/${f:3:2}/${f:6:4}/${f:11:2} && unzip $f -d ${f:0:2}/${f:3:2}/${f:6:4}/${f:11:2}/; done`
`rm */*/*.html && rm */*/*.zip && rm */*/*.md5`

On ordonne le tout pour que toutes les données soient dans data/TYPE_DONNEES/ANNEE/MOIS/JOUR/GREFFECODE/GREFFECODE2/DONNEESCSV

In [1]:
import sqlite3
import shutil
import zipfile
import glob
import unidecode
import os
import py7zr
from datetime import date, timedelta
import pandas as pd
import time
from os.path import exists


In [2]:
sdate = date(2017,5,4)   # start date
#sdate = date(2022,6,29)   # start date
edate = date(2022,9,3)   # end date
list_dates = [x.strftime("%Y-%m-%d") for x in pd.date_range(sdate,edate-timedelta(days=1),freq='d').to_list()]

In [3]:
vol_data = '/srv/geoff/inpi/data/'
vol_data = '/Volumes/OnlyData/tmp/'

In [4]:
# Etape de processing des données en amont du lancement de l'enrichissement de la bdd sqlite
def concatFilesRep(type_file, name_concat, pattern):
    pathdate = vol_data + type_file + "/" + ld.split('-')[0] + '/' + ld.split('-')[1] + '/' + ld.split('-')[2] + '/'
    consofile = vol_data + 'synthese/' + ld + name_concat + '.csv'
    # get list files stock
    list_files = glob.glob(pathdate + pattern)
    list_files.sort()
    if len(list_files) > 0:
        
        with open(consofile,"wb") as fout:
            # first file:
            with open(list_files[0], "rb") as f:
                fout.write(f.read())
            # now the rest:    
            for ls in list_files[1:]:
                with open(ls, "rb") as f:
                    try:
                        next(f) # skip the header
                    except:
                        pass
                    fout.write(f.read())
        
        
        #os.system('rm ' + consofile + ' && ulimit -n 10240 && head -n1 ' + list_files[0] + ' > ' + consofile)
        #for ls in list_files:
        #    os.system('tail -n+2 -q ' + ls + ' >> ' + consofile)
        
        print(name_concat + " ok")

In [9]:
for ld in list_dates:
    print(ld)
    concatFilesRep("stock", "_stock_rep", "*/*/*_5_rep.csv")
    concatFilesRep("flux-tc", "_flux_rep", "*/*/*_5_rep.csv")
    concatFilesRep("flux-tc", "_flux_rep_nouveau_modifie", "*/*/*6_rep_nouveau_modifie_EVT.csv")
    concatFilesRep("flux-tc", "_flux_rep_partant", "*/*/*7_rep_partant_EVT.csv")
    

2022-05-31
_flux_rep ok
_flux_rep_nouveau_modifie ok
_flux_rep_partant ok
2022-06-01
_flux_rep ok
_flux_rep_nouveau_modifie ok
_flux_rep_partant ok
2022-06-02
_flux_rep ok
_flux_rep_nouveau_modifie ok
_flux_rep_partant ok
2022-06-03
_flux_rep ok
_flux_rep_nouveau_modifie ok
_flux_rep_partant ok
2022-06-04
_flux_rep ok
_flux_rep_nouveau_modifie ok
_flux_rep_partant ok
2022-06-05
2022-06-06
2022-06-07
_flux_rep ok
_flux_rep_nouveau_modifie ok
_flux_rep_partant ok
2022-06-08
_flux_rep ok
_flux_rep_nouveau_modifie ok
_flux_rep_partant ok
2022-06-09
_flux_rep ok
_flux_rep_nouveau_modifie ok
_flux_rep_partant ok
2022-06-10
_flux_rep ok
_flux_rep_nouveau_modifie ok
_flux_rep_partant ok
2022-06-11
_flux_rep ok
_flux_rep_nouveau_modifie ok
_flux_rep_partant ok
2022-06-12
2022-06-13
2022-06-14
_flux_rep ok
_flux_rep_nouveau_modifie ok
_flux_rep_partant ok
2022-06-15
_flux_rep ok
_flux_rep_nouveau_modifie ok
_flux_rep_partant ok
2022-06-16
_flux_rep ok
_flux_rep_nouveau_modifie ok
_flux_rep_parta

In [54]:
!rm inpi.db

In [67]:
connection = sqlite3.connect('inpi.db')

In [68]:
cursor = connection.cursor()

In [57]:
cursor.execute('''CREATE TABLE IF NOT EXISTS rep_pp
              (siren TEXT, nom_patronymique TEXT, nom_usage TEXT, prenoms TEXT, datenaissance TEXT, villenaissance TEXT, paysnaissance TEXT, qualite TEXT)''')


In [58]:
cursor.execute('''CREATE TABLE IF NOT EXISTS rep_pm
              (siren TEXT, denomination TEXT, siren_pm TEXT, sigle TEXT, qualite TEXT)''')

In [59]:
cursor.execute('''CREATE INDEX idx_siren_pp ON rep_pp (siren);''')
cursor.execute('''CREATE INDEX idx_siren_pm ON rep_pm (siren);''')
cursor.execute('''CREATE INDEX idx_nom_patronymique_pp ON rep_pp (nom_patronymique);''')
cursor.execute('''CREATE INDEX idx_nom_usage_pp ON rep_pp (nom_usage);''')
cursor.execute('''CREATE INDEX idx_siren_pm_pm ON rep_pm (siren_pm);''')

In [60]:
connection.commit()

In [69]:
def uniformizeDf(df):
    for c in df.columns:
        df = df.rename(columns={c: unidecode.unidecode(
                                c.lower() \
                                .replace(' ', '') \
                                .replace('.','') \
                                .replace('_','') \
                                .replace('"','') \
                                .replace("'",'')
                    )})
    if 'sigle' not in df.columns:
        df['sigle'] = ''
    # Do something
    df['siren'] = df.fillna('')['siren']
    df['nom_patronymique'] = df.fillna('')['nompatronymique']
    df['nom_usage'] = df.fillna('')['nomusage']
    df['prenoms'] = df.fillna('')['prenoms']
    df['nom_patronymique'] = df['nom_patronymique'].apply(lambda x: str(x).replace(',','').lower())
    df['nom_usage'] = df['nom_usage'].apply(lambda x: str(x).replace(',','').lower())
    df['prenoms'] = df['prenoms'].apply(lambda x: str(x).replace(',','').lower())
    df['datenaissance'] = df.fillna('')['datenaissance']
    df['villenaissance'] = df.fillna('')['villenaissance']
    df['paysnaissance'] = df.fillna('')['paysnaissance']
    df['denomination'] = df.fillna('')['denomination']
    df['siren_pm'] = df.fillna('')['siren1']
    df['sigle'] = df.fillna('')['sigle']
    df['qualite'] = df.fillna('')['qualite']
    dfpp = df[df['type'].str.contains('Physique')][['siren', 'nom_patronymique', 'nom_usage','prenoms', 'datenaissance', 'villenaissance', 'paysnaissance', 'qualite']]
    dfpm = df[df['type'].str.contains('Morale')][['siren','denomination', 'siren_pm', 'sigle', 'qualite']]

    return dfpp, dfpm


In [62]:
# On s'occupe du stock initial (car vraiment gros et pas adapté à la méthode ci-dessous)
list_files = glob.glob(vol_data + 'stock/2017/05/04/*/*/*5_rep.csv')
list_files.sort()
for lf in list_files:
    print(lf)
    df = pd.read_csv(lf,sep=";",dtype=str)
    dfpp, dfpm = uniformizeDf(df)
    dfpp.set_index('siren').to_sql('rep_pp', con=connection, if_exists='append')
    dfpm.set_index('siren').to_sql('rep_pm', con=connection, if_exists='append')

/Volumes/OnlyData/tmp/stock/2017/05/04/0101/S1/0101_S1_20170504_5_rep.csv
/Volumes/OnlyData/tmp/stock/2017/05/04/0202/S1/0202_S1_20170504_5_rep.csv
/Volumes/OnlyData/tmp/stock/2017/05/04/0203/S1/0203_S1_20170504_5_rep.csv
/Volumes/OnlyData/tmp/stock/2017/05/04/0301/S1/0301_S1_20170504_5_rep.csv
/Volumes/OnlyData/tmp/stock/2017/05/04/0303/S1/0303_S1_20170504_5_rep.csv
/Volumes/OnlyData/tmp/stock/2017/05/04/0401/S1/0401_S1_20170504_5_rep.csv
/Volumes/OnlyData/tmp/stock/2017/05/04/0501/S1/0501_S1_20170504_5_rep.csv
/Volumes/OnlyData/tmp/stock/2017/05/04/0601/S1/0601_S1_20170504_5_rep.csv
/Volumes/OnlyData/tmp/stock/2017/05/04/0602/S1/0602_S1_20170504_5_rep.csv
/Volumes/OnlyData/tmp/stock/2017/05/04/0603/S1/0603_S1_20170504_5_rep.csv
/Volumes/OnlyData/tmp/stock/2017/05/04/0605/S1/0605_S1_20170504_5_rep.csv
/Volumes/OnlyData/tmp/stock/2017/05/04/0702/S1/0702_S1_20170504_5_rep.csv
/Volumes/OnlyData/tmp/stock/2017/05/04/0802/S1/0802_S1_20170504_5_rep.csv
/Volumes/OnlyData/tmp/stock/2017/05/04

/Volumes/OnlyData/tmp/stock/2017/05/04/7702/S1/7702_S1_20170504_5_rep.csv
/Volumes/OnlyData/tmp/stock/2017/05/04/7801/S1/7801_S1_20170504_5_rep.csv
/Volumes/OnlyData/tmp/stock/2017/05/04/7802/S1/7802_S1_20170504_5_rep.csv
/Volumes/OnlyData/tmp/stock/2017/05/04/7803/S1/7803_S1_20170504_5_rep.csv
/Volumes/OnlyData/tmp/stock/2017/05/04/7901/S1/7901_S1_20170504_5_rep.csv
/Volumes/OnlyData/tmp/stock/2017/05/04/8002/S1/8002_S1_20170504_5_rep.csv
/Volumes/OnlyData/tmp/stock/2017/05/04/8101/S1/8101_S1_20170504_5_rep.csv
/Volumes/OnlyData/tmp/stock/2017/05/04/8102/S1/8102_S1_20170504_5_rep.csv
/Volumes/OnlyData/tmp/stock/2017/05/04/8201/S1/8201_S1_20170504_5_rep.csv
/Volumes/OnlyData/tmp/stock/2017/05/04/8302/S1/8302_S1_20170504_5_rep.csv
/Volumes/OnlyData/tmp/stock/2017/05/04/8303/S1/8303_S1_20170504_5_rep.csv
/Volumes/OnlyData/tmp/stock/2017/05/04/8305/S1/8305_S1_20170504_5_rep.csv
/Volumes/OnlyData/tmp/stock/2017/05/04/8401/S1/8401_S1_20170504_5_rep.csv
/Volumes/OnlyData/tmp/stock/2017/05/04

In [63]:
original = 'inpi.db'
target = 'inpi-stock-initial.db'
shutil.copyfile(original, target)

'inpi-stock-initial.db'

In [79]:
list_dates[1368]

'2021-01-31'

In [65]:
len(list_dates)

1948

In [80]:
# Toutes les dates après la date du stock initial
for ld in list_dates[1368:]:
    print(ld)
    #if((ld.split('-')[1] == '01') & (ld.split('-')[2] == '01')):
    if(ld.split('-')[2] == '01'):
        original = 'inpi.db'
        target = 'inpi-'+ld+'.db'
        shutil.copyfile(original, target)
    
    consofile = vol_data + 'synthese/' + ld + '_stock_rep.csv'
    # Manage stocks
    if(os.path.exists(consofile)):
        df = pd.read_csv(consofile,sep=";",dtype=str,on_bad_lines='warn')
        dfpp, dfpm = uniformizeDf(df)
        print('loaded and uniformize')
        # Delete old stock
        if(ld != '2017-05-04'):
            print('Old stock')
            result = 0
            for index,row in dfpp.iterrows():
                del_query = """DELETE from rep_pp where siren = ?"""
                cursor.execute(del_query, (row['siren'],))
                result = result + cursor.rowcount
            connection.commit()
            for index,row in dfpp.iterrows():
                del_query = """DELETE from rep_pm where siren = ?"""
                cursor.execute(del_query, (row['siren'],))
                result = result + cursor.rowcount
            connection.commit()
            print('Old Stock deleted : ' + str(result) + ' deleted records')
        # Add new stock
        dfpp.set_index('siren').to_sql('rep_pp', con=connection, if_exists='append')
        dfpm.set_index('siren').to_sql('rep_pm', con=connection, if_exists='append')
        print('Stock processed : ' + str(dfpp.shape[0]+dfpm.shape[0]) + 'added records')

    
    consofile = vol_data + 'synthese/' + ld + '_flux_rep.csv'
    if(os.path.exists(consofile)):
        df = pd.read_csv(consofile,sep=";",dtype=str,on_bad_lines='warn')
        dfpp, dfpm = uniformizeDf(df)
        dfpp.set_index('siren').to_sql('rep_pp', con=connection, if_exists='append')
        dfpm.set_index('siren').to_sql('rep_pm', con=connection, if_exists='append')
        print('Flux rep processed : ' + str(dfpp.shape[0]+dfpm.shape[0]) + 'added records')

    consofile = vol_data + 'synthese/' + ld + '_flux_rep_nouveau_modifie.csv'
    if(os.path.exists(consofile)):
        df = pd.read_csv(consofile,sep=";",dtype=str,on_bad_lines='warn')
        dfpp, dfpm = uniformizeDf(df)
        dfpp.set_index('siren').to_sql('rep_pp', con=connection, if_exists='append')
        dfpm.set_index('siren').to_sql('rep_pm', con=connection, if_exists='append')
        print('Flux rep modified new processed : ' + str(dfpp.shape[0]+dfpm.shape[0]) + 'added records')

    
    consofile = vol_data + 'synthese/' + ld + '_flux_rep_partant.csv'
    if(os.path.exists(consofile)):
        df = pd.read_csv(consofile,sep=";",dtype=str,on_bad_lines='warn')
        dfpp, dfpm = uniformizeDf(df)
        # Delete each rep
        result = 0
        for index,row in dfpp.iterrows():
            del_query = """DELETE from rep_pp where siren = ? AND nom_patronymique = ? AND prenoms = ? AND qualite = ?"""
            cursor.execute(del_query, (row['siren'], row['nom_patronymique'], row['prenoms'], row['qualite']))
            result = result + cursor.rowcount
        connection.commit()
        for index,row in dfpm.iterrows():
            del_query = """DELETE from rep_pm where siren = ? AND siren_pm = ? AND qualite = ?"""
            cursor.execute(del_query, (row['siren'], row['siren_pm'], row['qualite']))
            result = result + cursor.rowcount
        connection.commit()
        print('Flux rep partant processed : ' + str(result) + ' deleted records')

        


2021-01-31
2021-02-01
loaded and uniformize
Old stock
Old Stock deleted : 599 deleted records
Stock processed : 59674added records
Flux rep processed : 530added records
Flux rep modified new processed : 559added records
Flux rep partant processed : 209 deleted records
2021-02-02
Flux rep processed : 5457added records
Flux rep modified new processed : 15968added records
Flux rep partant processed : 2732 deleted records
2021-02-03
Flux rep processed : 4428added records
Flux rep modified new processed : 4194added records
Flux rep partant processed : 1645 deleted records
2021-02-04
Flux rep processed : 4194added records
Flux rep modified new processed : 5116added records
Flux rep partant processed : 1879 deleted records
2021-02-05
Flux rep processed : 3869added records
Flux rep modified new processed : 4626added records
Flux rep partant processed : 1940 deleted records
2021-02-06
Flux rep processed : 3594added records
Flux rep modified new processed : 3913added records
Flux rep partant pro

Flux rep processed : 4478added records
Flux rep modified new processed : 3973added records
Flux rep partant processed : 1520 deleted records
2021-04-10
Flux rep processed : 3608added records
Flux rep modified new processed : 3643added records
Flux rep partant processed : 1810 deleted records
2021-04-11
Flux rep processed : 3893added records
Flux rep modified new processed : 3345added records
Flux rep partant processed : 1369 deleted records
2021-04-12
loaded and uniformize
Old stock
Old Stock deleted : 44069 deleted records
Stock processed : 23630added records
Flux rep processed : 97added records
Flux rep modified new processed : 119added records
Flux rep partant processed : 43 deleted records
2021-04-13
Flux rep processed : 642added records
Flux rep modified new processed : 922added records
Flux rep partant processed : 394 deleted records
2021-04-14
Flux rep processed : 4053added records
Flux rep modified new processed : 4089added records
Flux rep partant processed : 1599 deleted reco

Flux rep processed : 8817added records
Flux rep modified new processed : 7597added records
Flux rep partant processed : 3107 deleted records
2021-06-15
Flux rep processed : 3758added records
Flux rep modified new processed : 3121added records
Flux rep partant processed : 1189 deleted records
2021-06-16
Flux rep processed : 7053added records
Flux rep modified new processed : 5391added records
Flux rep partant processed : 1986 deleted records
2021-06-17
Flux rep processed : 2901added records
Flux rep modified new processed : 2548added records
Flux rep partant processed : 859 deleted records
2021-06-18
Flux rep processed : 9668added records
Flux rep modified new processed : 8693added records
Flux rep partant processed : 3392 deleted records
2021-06-19
Flux rep processed : 2696added records
Flux rep modified new processed : 2736added records
Flux rep partant processed : 1095 deleted records
2021-06-20
2021-06-21
Flux rep processed : 2568added records
Flux rep modified new processed : 2092a

Flux rep partant processed : 1324 deleted records
2021-08-20
Flux rep processed : 2504added records
Flux rep modified new processed : 2928added records
Flux rep partant processed : 1240 deleted records
2021-08-21
Flux rep processed : 1802added records
Flux rep modified new processed : 2343added records
Flux rep partant processed : 933 deleted records
2021-08-22
2021-08-23
2021-08-24
Flux rep processed : 2521added records
Flux rep modified new processed : 3319added records
Flux rep partant processed : 1364 deleted records
2021-08-25
Flux rep processed : 2207added records
Flux rep modified new processed : 3283added records
Flux rep partant processed : 1367 deleted records
2021-08-26
Flux rep processed : 2211added records
Flux rep modified new processed : 3154added records
Flux rep partant processed : 1568 deleted records
2021-08-27
Flux rep processed : 2460added records
Flux rep modified new processed : 3196added records
Flux rep partant processed : 1160 deleted records
2021-08-28
Flux r

Flux rep modified new processed : 4649added records
Flux rep partant processed : 1083 deleted records
2021-11-04
Flux rep processed : 4003added records
Flux rep modified new processed : 3738added records
Flux rep partant processed : 897 deleted records
2021-11-05
Flux rep processed : 4637added records
Flux rep modified new processed : 4506added records
Flux rep partant processed : 875 deleted records
2021-11-06
Flux rep processed : 2935added records
Flux rep modified new processed : 3483added records
Flux rep partant processed : 820 deleted records
2021-11-07
2021-11-08
2021-11-09
Flux rep processed : 5028added records
Flux rep modified new processed : 5018added records
Flux rep partant processed : 1139 deleted records
2021-11-10
Flux rep processed : 3819added records
Flux rep modified new processed : 4367added records
Flux rep partant processed : 999 deleted records
2021-11-11
Flux rep processed : 4108added records
Flux rep modified new processed : 3435added records
Flux rep partant p

Flux rep processed : 3798added records
Flux rep modified new processed : 3992added records
Flux rep partant processed : 701 deleted records
2022-01-16
2022-01-17
2022-01-18
Flux rep processed : 4889added records
Flux rep modified new processed : 5540added records
Flux rep partant processed : 995 deleted records
2022-01-19
Flux rep processed : 4199added records
Flux rep modified new processed : 5714added records
Flux rep partant processed : 1084 deleted records
2022-01-20
Flux rep processed : 3847added records
Flux rep modified new processed : 5008added records
Flux rep partant processed : 1815 deleted records
2022-01-21
Flux rep processed : 4253added records
Flux rep modified new processed : 5529added records
Flux rep partant processed : 1892 deleted records
2022-01-22
Flux rep processed : 4446added records
Flux rep modified new processed : 4341added records
Flux rep partant processed : 1628 deleted records
2022-01-23
2022-01-24
2022-01-25
loaded and uniformize
Old stock
Old Stock dele

Flux rep processed : 4018added records
Flux rep modified new processed : 4472added records
Flux rep partant processed : 659 deleted records
2022-04-01
Flux rep processed : 4094added records
Flux rep modified new processed : 4359added records
Flux rep partant processed : 710 deleted records
2022-04-02
Flux rep processed : 3734added records
Flux rep modified new processed : 3581added records
Flux rep partant processed : 575 deleted records
2022-04-03
2022-04-04
2022-04-05
loaded and uniformize
Old stock
Old Stock deleted : 103531 deleted records
Stock processed : 52768added records
2022-04-06
loaded and uniformize
Old stock
Old Stock deleted : 31324 deleted records
Stock processed : 25318added records
Flux rep processed : 624added records
Flux rep modified new processed : 1063added records
Flux rep partant processed : 427 deleted records
2022-04-07
Flux rep processed : 14091added records
Flux rep modified new processed : 17013added records
Flux rep partant processed : 3479 deleted record

Flux rep processed : 4305added records
Flux rep modified new processed : 4512added records
Flux rep partant processed : 2014 deleted records
2022-06-11
Flux rep processed : 3582added records
Flux rep modified new processed : 3495added records
Flux rep partant processed : 1508 deleted records
2022-06-12
2022-06-13
2022-06-14
Flux rep processed : 5480added records
Flux rep modified new processed : 5560added records
Flux rep partant processed : 2081 deleted records
2022-06-15
Flux rep processed : 4616added records
Flux rep modified new processed : 5145added records
Flux rep partant processed : 1921 deleted records
2022-06-16
Flux rep processed : 4279added records
Flux rep modified new processed : 4574added records
Flux rep partant processed : 1517 deleted records
2022-06-17
Flux rep processed : 3411added records
Flux rep modified new processed : 4042added records
Flux rep partant processed : 1607 deleted records
2022-06-18
Flux rep processed : 3650added records
Flux rep modified new proce

Flux rep processed : 2734added records
Flux rep modified new processed : 3657added records
Flux rep partant processed : 1416 deleted records
2022-08-24
Flux rep processed : 2364added records
Flux rep modified new processed : 3647added records
Flux rep partant processed : 1474 deleted records
2022-08-25
Flux rep processed : 2514added records
Flux rep modified new processed : 3251added records
Flux rep partant processed : 1555 deleted records
2022-08-26
Flux rep processed : 2397added records
Flux rep modified new processed : 3337added records
Flux rep partant processed : 1421 deleted records
2022-08-27
Flux rep processed : 2107added records
Flux rep modified new processed : 2904added records
Flux rep partant processed : 1181 deleted records
2022-08-28
2022-08-29
2022-08-30
Flux rep processed : 3074added records
Flux rep modified new processed : 4004added records
Flux rep partant processed : 1704 deleted records
2022-08-31
Flux rep processed : 2786added records
Flux rep modified new proce

In [83]:
conn = sqlite3.connect('inpi.db', isolation_level=None,
                       detect_types=sqlite3.PARSE_COLNAMES)
print('connected')

db = pd.read_sql_query("SELECT * FROM rep_pm", conn)
print('read rep_pm ok')
db = db.drop_duplicates(keep="first")
print('deduplicates ok')
dbpm = db.groupby(['siren','denomination', 'siren_pm'],as_index=False)['qualite'].agg(lambda col: ', '.join(col))
print('groupby ok')
dbpm[dbpm['siren'] != ''].to_csv('rep_pm.csv',index=False)
print('csv saved ok')

db = pd.read_sql_query("SELECT * FROM rep_pp", conn)
print('read rep_pp ok')

inter = db[db['villenaissance'].notna()]
inter = inter[['siren', 'nom_patronymique', 'nom_usage', 'prenoms', 'villenaissance']]
inter = inter.drop_duplicates(keep="first")
db = db.drop(['villenaissance'], axis=1)
db = pd.merge(db, inter, on=['siren', 'nom_patronymique', 'nom_usage', 'prenoms'],how='left')
print('deduplicates villenaissance ok')

inter = db[db['paysnaissance'].notna()]
inter = inter[['siren', 'nom_patronymique', 'nom_usage', 'prenoms', 'paysnaissance']]
inter = inter.drop_duplicates(keep="first")
db = db.drop(['paysnaissance'], axis=1)
db = pd.merge(db, inter, on=['siren', 'nom_patronymique', 'nom_usage', 'prenoms'],how='left')
print('deduplicates paysnaissance ok')

inter = db[db['datenaissance'].notna()]
inter = inter[['siren', 'nom_patronymique', 'nom_usage', 'prenoms', 'datenaissance']]
inter = inter.drop_duplicates(keep="first")
db = db.drop(['datenaissance'], axis=1)
db = pd.merge(db, inter, on=['siren', 'nom_patronymique', 'nom_usage', 'prenoms'],how='left')
print('deduplicates datenaissance ok')

db = db.drop_duplicates(keep="first")
print('deduplicates ok')
dbpp = db.groupby(['siren','nom_patronymique', 'nom_usage','prenoms', 'villenaissance', 'paysnaissance', 'datenaissance'],as_index=False)['qualite'].agg(lambda col: ', '.join(col))
print('groupby ok')
dbpp[dbpp['siren'] != ''].to_csv('rep_pp.csv',index=False)
print('csv saved ok') 


connected
read rep_pm ok
deduplicates ok
groupby ok
csv saved ok
read rep_pp ok
deduplicates ok
groupby ok
csv saved ok
